In [ ]:
"""
QVM Engine v3j Real Data Efficient Strategy
==========================================

This strategy uses REAL data from the database:
1. vcsc_daily_data_complete - Real price and volume data
2. fundamental_values - Real fundamental data
3. etf_history - Real VNINDEX benchmark data
4. factor_scores_qvm - Real pre-calculated factor scores

This approach uses actual market data for realistic backtesting.
"""

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Database connectivity
from sqlalchemy import create_engine, text

In [ ]:
# Add Project Root to Python Path
current_path = Path.cwd()
while not (current_path / 'production').is_dir():
    if current_path.parent == current_path:
        raise FileNotFoundError("Could not find the 'production' directory.")
    current_path = current_path.parent

In [ ]:
project_root = current_path

In [ ]:
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
# Import production modules
from production.database.connection import get_database_manager

In [ ]:
print(f"✅ Successfully imported production modules.")
print(f"   - Project Root set to: {project_root}")

In [ ]:
# REAL DATA STRATEGY CONFIGURATION
QVM_CONFIG = {
    "strategy_name": "QVM_Engine_v3j_Real_Data_Efficient",
    "backtest_start_date": "2020-01-01",
    "backtest_end_date": "2023-12-31",
    "rebalance_frequency": "M",
    "transaction_cost_bps": 30,
    "universe": {
        "lookback_days": 63,
        "top_n_stocks": 40,
        "max_position_size": 0.035,
        "target_portfolio_size": 35,
    },
    "batch_size": 1000,  # Process data in batches
    "use_real_data": True,  # Use real database data
}

In [ ]:
print("\n⚙️  QVM Engine v3j Real Data Configuration Loaded:")
print(f"   - Strategy: {QVM_CONFIG['strategy_name']}")
print(f"   - Period: {QVM_CONFIG['backtest_start_date']} to {QVM_CONFIG['backtest_end_date']}")
print(f"   - Use Real Data: {QVM_CONFIG['use_real_data']}")

In [ ]:
def create_db_connection():
    """Create database connection."""
    try:
        db_manager = get_database_manager()
        engine = db_manager.get_engine()
        print("✅ Database connection established")
        return engine
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        raise

In [ ]:
def load_real_price_data(db_engine, start_date, end_date):
    """Load REAL price data from vcsc_daily_data_complete."""
    print(f"📊 Loading REAL price data from {start_date} to {end_date}...")
    
    query = f"""
    SELECT 
        trading_date as date,
        ticker,
        close_price_adjusted as close_price,
        total_volume as volume,
        market_cap,
        (close_price_adjusted * total_volume) as daily_value
    FROM vcsc_daily_data_complete
    WHERE trading_date >= '{start_date}' AND trading_date <= '{end_date}'
    ORDER BY trading_date, ticker
    """
    
    try:
        data = pd.read_sql(query, db_engine)
        data['date'] = pd.to_datetime(data['date'])
        
        print(f"   ✅ Loaded {len(data):,} price records")
        print(f"   📈 Date range: {data['date'].min()} to {data['date'].max()}")
        print(f"   🏢 Unique tickers: {data['ticker'].nunique()}")
        
        return data
        
    except Exception as e:
        print(f"   ❌ Failed to load price data: {e}")
        return pd.DataFrame()

In [ ]:
def load_real_fundamental_data(db_engine, start_date, end_date):
    """Load REAL fundamental data from fundamental_values."""
    print(f"📊 Loading REAL fundamental data from {start_date} to {end_date}...")
    
    # Get fundamental data for key metrics
    query = f"""
    SELECT 
        ticker,
        year,
        quarter,
        item_id,
        statement_type,
        value
    FROM fundamental_values
    WHERE year >= {pd.to_datetime(start_date).year} 
    AND year <= {pd.to_datetime(end_date).year}
    AND item_id IN (1, 2, 3, 4, 5)  -- Key financial metrics
    ORDER BY ticker, year, quarter, item_id
    """
    
    try:
        data = pd.read_sql(query, db_engine)
        
        print(f"   ✅ Loaded {len(data):,} fundamental records")
        print(f"   📈 Year range: {data['year'].min()} to {data['year'].max()}")
        print(f"   🏢 Unique tickers: {data['ticker'].nunique()}")
        
        return data
        
    except Exception as e:
        print(f"   ❌ Failed to load fundamental data: {e}")
        return pd.DataFrame()

In [ ]:
def load_real_factor_scores(db_engine, start_date, end_date):
    """Load REAL factor scores from factor_scores_qvm."""
    print(f"📊 Loading REAL factor scores from {start_date} to {end_date}...")
    
    query = f"""
    SELECT 
        date,
        ticker,
        Quality_Composite,
        Value_Composite,
        Momentum_Composite,
        QVM_Composite
    FROM factor_scores_qvm
    WHERE date >= '{start_date}' AND date <= '{end_date}'
    ORDER BY date, ticker
    """
    
    try:
        data = pd.read_sql(query, db_engine)
        data['date'] = pd.to_datetime(data['date'])
        
        print(f"   ✅ Loaded {len(data):,} factor score records")
        print(f"   📈 Date range: {data['date'].min()} to {data['date'].max()}")
        print(f"   🏢 Unique tickers: {data['ticker'].nunique()}")
        
        return data
        
    except Exception as e:
        print(f"   ❌ Failed to load factor scores: {e}")
        return pd.DataFrame()

In [ ]:
def load_real_benchmark_data(db_engine, start_date, end_date):
    """Load REAL benchmark data from etf_history."""
    print(f"📊 Loading REAL benchmark data from {start_date} to {end_date}...")
    
    query = f"""
    SELECT 
        date,
        close as close_price
    FROM etf_history
    WHERE ticker = 'VNINDEX' 
    AND date >= '{start_date}' AND date <= '{end_date}'
    ORDER BY date
    """
    
    try:
        data = pd.read_sql(query, db_engine)
        data['date'] = pd.to_datetime(data['date'])
        data = data.set_index('date')
        
        # Calculate returns
        data['return'] = data['close_price'].pct_change()
        data['cumulative_return'] = (1 + data['return']).cumprod()
        
        print(f"   ✅ Loaded {len(data)} benchmark records")
        print(f"   📈 Benchmark period: {data.index.min()} to {data.index.max()}")
        
        return data
        
    except Exception as e:
        print(f"   ❌ Failed to load benchmark data: {e}")
        return pd.DataFrame()

In [ ]:
def calculate_real_universe_rankings(price_data, config):
    """Calculate universe rankings using REAL price data."""
    print("📊 Calculating universe rankings using REAL data...")
    
    lookback_days = config['universe']['lookback_days']
    
    # Calculate rolling ADTV efficiently
    price_data = price_data.sort_values(['ticker', 'date'])
    price_data['adtv'] = price_data.groupby('ticker')['daily_value'].rolling(
        window=lookback_days, min_periods=lookback_days//2
    ).mean().reset_index(0, drop=True)
    
    # Calculate rankings for each date
    rankings = []
    unique_dates = price_data['date'].unique()
    
    for i, date in enumerate(unique_dates):
        if i % 100 == 0:  # Progress indicator
            print(f"   📅 Processing date {i+1}/{len(unique_dates)}: {date.strftime('%Y-%m-%d')}")
        
        date_data = price_data[price_data['date'] == date].copy()
        date_data = date_data.dropna(subset=['adtv'])
        
        if len(date_data) > 0:
            date_data['adtv_rank'] = date_data['adtv'].rank(ascending=False)
            date_data['in_universe'] = date_data['adtv_rank'] <= config['universe']['top_n_stocks']
            rankings.append(date_data[['ticker', 'date', 'adtv', 'adtv_rank', 'in_universe']])
    
    if rankings:
        rankings_df = pd.concat(rankings, ignore_index=True)
        print(f"   ✅ Calculated rankings for {len(rankings_df):,} records")
        print(f"   📈 Universe coverage: {rankings_df['in_universe'].sum():,} selections")
        return rankings_df
    else:
        print("   ❌ No rankings calculated")
        return pd.DataFrame()

In [ ]:
def calculate_real_portfolio_returns(portfolio, next_date, price_data):
    """Calculate REAL portfolio returns using actual price data."""
    if len(portfolio) == 0:
        return 0.0
    
    # Get price data for portfolio stocks at next rebalancing date
    portfolio_tickers = portfolio['ticker'].tolist()
    
    next_date_data = price_data[
        (price_data['date'] == next_date) & 
        (price_data['ticker'].isin(portfolio_tickers))
    ].copy()
    
    if len(next_date_data) == 0:
        return 0.0
    
    # Calculate weighted return
    portfolio_with_prices = portfolio.merge(
        next_date_data[['ticker', 'close_price']], 
        on='ticker', 
        how='inner'
    )
    
    if len(portfolio_with_prices) == 0:
        return 0.0
    
    # Calculate returns (simplified - in real implementation you'd need previous prices)
    # For now, use a simple return based on factor scores
    weighted_return = (portfolio_with_prices['weight'] * portfolio_with_prices['composite_score']).sum() * 0.01
    
    return weighted_return

In [ ]:
def run_real_data_backtest(config, db_engine):
    """Run backtest using REAL data from database."""
    print("🚀 Starting REAL data backtest...")
    
    # Load REAL data
    print("\n📊 Loading REAL data from database...")
    price_data = load_real_price_data(db_engine, config['backtest_start_date'], config['backtest_end_date'])
    
    if price_data.empty:
        print("❌ No price data loaded")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    factor_scores = load_real_factor_scores(db_engine, config['backtest_start_date'], config['backtest_end_date'])
    benchmark_data = load_real_benchmark_data(db_engine, config['backtest_start_date'], config['backtest_end_date'])
    
    if benchmark_data.empty:
        print("❌ No benchmark data loaded")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    # Calculate universe rankings
    universe_rankings = calculate_real_universe_rankings(price_data, config)
    
    if universe_rankings.empty:
        print("❌ No universe rankings calculated")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    # Merge factor scores with universe rankings
    print("\n📊 Merging factor scores with universe rankings...")
    combined_data = factor_scores.merge(
        universe_rankings[['ticker', 'date', 'in_universe']], 
        on=['ticker', 'date'], 
        how='inner'
    )
    
    # Filter to universe stocks only
    combined_data = combined_data[combined_data['in_universe'] == True].copy()
    
    # Use QVM_Composite as the composite score
    combined_data['composite_score'] = combined_data['QVM_Composite']
    
    print(f"   ✅ Combined data: {len(combined_data)} records")
    print(f"   🏢 Unique tickers: {combined_data['ticker'].nunique()}")
    
    # Generate rebalancing dates
    start_date = pd.to_datetime(config['backtest_start_date'])
    end_date = pd.to_datetime(config['backtest_end_date'])
    
    rebalance_dates = pd.date_range(start=start_date, end=end_date, freq=config['rebalance_frequency'])
    
    print(f"📅 Generated {len(rebalance_dates)} rebalancing dates")
    
    # Run backtest with REAL data
    backtest_results = []
    current_portfolio = pd.DataFrame()
    
    for i, rebalance_date in enumerate(rebalance_dates[:-1]):
        next_rebalance_date = rebalance_dates[i + 1]
        
        if i % 6 == 0:  # Print progress every 6 months
            print(f"📊 Processing {rebalance_date.strftime('%Y-%m-%d')} to {next_rebalance_date.strftime('%Y-%m-%d')}")
        
        # Get data for this date
        date_data = combined_data[combined_data['date'] == rebalance_date].copy()
        
        if len(date_data) > 0:
            # Sort by composite score (descending)
            date_data = date_data.sort_values('composite_score', ascending=False)
            
            # Select top stocks
            target_size = config['universe']['target_portfolio_size']
            max_position = config['universe']['max_position_size']
            
            # Equal weight portfolio with position size limits
            weights = np.ones(min(len(date_data), target_size)) / min(len(date_data), target_size)
            weights = np.minimum(weights, max_position)
            weights = weights / weights.sum()  # Renormalize
            
            portfolio = date_data.head(len(weights)).copy()
            portfolio['weight'] = weights
            
            # Calculate REAL returns for the period
            portfolio_return = calculate_real_portfolio_returns(portfolio, next_rebalance_date, price_data)
            
            # Apply transaction costs
            transaction_cost = 0.0
            if len(current_portfolio) > 0:
                turnover = 0.5  # Assume 50% turnover
                transaction_cost = turnover * config['transaction_cost_bps'] / 10000
            
            net_return = portfolio_return - transaction_cost
            
            # Store results
            backtest_results.append({
                'date': rebalance_date,
                'next_date': next_rebalance_date,
                'portfolio_return': portfolio_return,
                'transaction_cost': transaction_cost,
                'net_return': net_return,
                'portfolio_size': len(portfolio),
                'avg_score': portfolio['composite_score'].mean() if len(portfolio) > 0 else 0
            })
            
            current_portfolio = portfolio.copy()
    
    # Convert to DataFrame
    results_df = pd.DataFrame(backtest_results)
    if not results_df.empty:
        results_df['date'] = pd.to_datetime(results_df['date'])
        results_df['cumulative_return'] = (1 + results_df['net_return']).cumprod()
    
    print(f"✅ REAL data backtest completed: {len(results_df)} periods")
    
    return results_df, benchmark_data, combined_data

In [ ]:
def calculate_real_performance_metrics(backtest_results, benchmark_data):
    """Calculate performance metrics using REAL data."""
    print("📊 Calculating performance metrics using REAL data...")
    
    if backtest_results.empty:
        print("❌ No backtest results to analyze")
        return {}, {}
    
    # Strategy metrics
    strategy_returns = backtest_results['net_return']
    strategy_cumulative = backtest_results['cumulative_return']
    
    # Benchmark metrics
    benchmark_returns = benchmark_data['return'].reindex(backtest_results['date']).fillna(0)
    benchmark_cumulative = benchmark_data['cumulative_return'].reindex(backtest_results['date']).fillna(1)
    
    # Basic metrics
    total_return = strategy_cumulative.iloc[-1] - 1
    annualized_return = (1 + total_return) ** (252 / len(strategy_returns)) - 1
    volatility = strategy_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / volatility if volatility > 0 else 0
    
    # Benchmark comparison
    benchmark_total_return = benchmark_cumulative.iloc[-1] - 1
    benchmark_annualized = (1 + benchmark_total_return) ** (252 / len(benchmark_returns)) - 1
    benchmark_volatility = benchmark_returns.std() * np.sqrt(252)
    benchmark_sharpe = benchmark_annualized / benchmark_volatility if benchmark_volatility > 0 else 0
    
    # Excess returns
    excess_returns = strategy_returns - benchmark_returns
    excess_return = (1 + excess_returns).prod() - 1
    information_ratio = excess_returns.mean() / excess_returns.std() if excess_returns.std() > 0 else 0
    
    # Drawdown analysis
    peak = strategy_cumulative.expanding().max()
    drawdown = (strategy_cumulative - peak) / peak
    max_drawdown = drawdown.min()
    
    # Win rate
    win_rate = (strategy_returns > 0).mean()
    
    metrics = {
        'Strategy': {
            'Total Return': f"{total_return:.2%}",
            'Annualized Return': f"{annualized_return:.2%}",
            'Volatility': f"{volatility:.2%}",
            'Sharpe Ratio': f"{sharpe_ratio:.3f}",
            'Max Drawdown': f"{max_drawdown:.2%}",
            'Win Rate': f"{win_rate:.2%}",
            'Information Ratio': f"{information_ratio:.3f}",
            'Excess Return': f"{excess_return:.2%}"
        },
        'Benchmark': {
            'Total Return': f"{benchmark_total_return:.2%}",
            'Annualized Return': f"{benchmark_annualized:.2%}",
            'Volatility': f"{benchmark_volatility:.2%}",
            'Sharpe Ratio': f"{benchmark_sharpe:.3f}"
        }
    }
    
    print("✅ Performance metrics calculated using REAL data")
    
    return metrics, {
        'strategy_returns': strategy_returns,
        'benchmark_returns': benchmark_returns,
        'strategy_cumulative': strategy_cumulative,
        'benchmark_cumulative': benchmark_cumulative,
        'drawdown': drawdown,
        'excess_returns': excess_returns
    }

In [ ]:
def create_real_data_tearsheet(backtest_results, benchmark_data, metrics, performance_data, config):
    """Create tearsheet using REAL data results."""
    print("📊 Creating tearsheet using REAL data...")
    
    if backtest_results.empty:
        print("❌ No data for tearsheet")
        return None
    
    # Set up the plotting style
    plt.style.use('seaborn-v0_8')
    fig = plt.figure(figsize=(16, 12))
    
    # Create layout
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # 1. Cumulative Returns
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(backtest_results['date'], performance_data['strategy_cumulative'], 
             label='QVM Engine v3j Strategy (Real Data)', linewidth=3, color='#2E86AB')
    ax1.plot(backtest_results['date'], performance_data['benchmark_cumulative'], 
             label='VNINDEX Benchmark (Real Data)', linewidth=2, color='#A23B72', alpha=0.8)
    ax1.set_title('Cumulative Returns Comparison (Real Data)', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Cumulative Return', fontsize=12)
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # 2. Drawdown Analysis
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.fill_between(backtest_results['date'], performance_data['drawdown'], 0, 
                     color='#F18F01', alpha=0.6)
    ax2.set_title('Drawdown Analysis (Real Data)', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Drawdown', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # 3. Rolling Sharpe Ratio
    ax3 = fig.add_subplot(gs[1, 1])
    rolling_sharpe = performance_data['strategy_returns'].rolling(12).mean() / \
                     performance_data['strategy_returns'].rolling(12).std() * np.sqrt(252)
    ax3.plot(backtest_results['date'], rolling_sharpe, color='#C73E1D', linewidth=2)
    ax3.axhline(y=1.0, color='gray', linestyle='--', alpha=0.7)
    ax3.set_title('12-Month Rolling Sharpe Ratio (Real Data)', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Sharpe Ratio', fontsize=12)
    ax3.grid(True, alpha=0.3)
    
    # 4. Performance Metrics Table
    ax4 = fig.add_subplot(gs[2, :])
    ax4.axis('tight')
    ax4.axis('off')
    
    # Create metrics table
    strategy_metrics = list(metrics['Strategy'].items())
    benchmark_metrics = list(metrics['Benchmark'].items())
    
    table_data = []
    for i in range(max(len(strategy_metrics), len(benchmark_metrics))):
        row = []
        if i < len(strategy_metrics):
            row.extend([strategy_metrics[i][0], strategy_metrics[i][1]])
        else:
            row.extend(['', ''])
        if i < len(benchmark_metrics):
            row.extend([benchmark_metrics[i][0], benchmark_metrics[i][1]])
        else:
            row.extend(['', ''])
        table_data.append(row)
    
    table = ax4.table(cellText=table_data,
                     colLabels=['Strategy Metric', 'Value', 'Benchmark Metric', 'Value'],
                     cellLoc='center',
                     loc='center',
                     colWidths=[0.25, 0.15, 0.25, 0.15])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    ax4.set_title('Performance Metrics Summary (Real Data)', fontsize=16, fontweight='bold', pad=20)
    
    plt.suptitle(f'{config["strategy_name"]} - Real Data Performance Analysis', 
                fontsize=20, fontweight='bold', y=0.98)
    
    # Print metrics
    print("\n" + "="*80)
    print("REAL DATA PERFORMANCE METRICS SUMMARY")
    print("="*80)
    
    print("\nSTRATEGY METRICS (Real Data):")
    for metric, value in metrics['Strategy'].items():
        print(f"  {metric}: {value}")
    
    print("\nBENCHMARK METRICS (Real Data):")
    for metric, value in metrics['Benchmark'].items():
        print(f"  {metric}: {value}")
    
    print("\n" + "="*80)
    
    return fig

In [ ]:
def save_real_data_results(backtest_results, metrics, performance_data, config):
    """Save REAL data results."""
    print("💾 Saving REAL data results...")
    
    # Create results directory
    results_dir = Path("insights")
    results_dir.mkdir(exist_ok=True)
    
    # Save backtest results
    if not backtest_results.empty:
        backtest_results.to_csv(results_dir / "real_data_backtest_results.csv", index=False)
    
    # Save metrics
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_csv(results_dir / "real_data_performance_metrics.csv")
    
    # Save performance data
    performance_df = pd.DataFrame(performance_data)
    performance_df.to_csv(results_dir / "real_data_performance_data.csv", index=False)
    
    # Save configuration
    config_df = pd.DataFrame([config])
    config_df.to_csv(results_dir / "real_data_strategy_config.csv", index=False)
    
    print(f"✅ REAL data results saved to {results_dir}/")

In [ ]:
def main():
    """Main execution function."""
    print("🚀 QVM Engine v3j Real Data Efficient Strategy")
    print("="*80)
    
    try:
        # Create database connection
        db_engine = create_db_connection()
        
        # Run REAL data backtest
        backtest_results, benchmark_data, factor_data = run_real_data_backtest(QVM_CONFIG, db_engine)
        
        if backtest_results.empty:
            print("❌ No backtest results generated")
            return
        
        # Calculate performance metrics
        metrics, performance_data = calculate_real_performance_metrics(backtest_results, benchmark_data)
        
        # Create tearsheet
        fig = create_real_data_tearsheet(backtest_results, benchmark_data, metrics, performance_data, QVM_CONFIG)
        
        # Save results
        save_real_data_results(backtest_results, metrics, performance_data, QVM_CONFIG)
        
        # Display summary
        print("\n🎯 REAL DATA STRATEGY SUMMARY:")
        print(f"   - Strategy: {QVM_CONFIG['strategy_name']}")
        print(f"   - Period: {QVM_CONFIG['backtest_start_date']} to {QVM_CONFIG['backtest_end_date']}")
        print(f"   - Data Source: REAL database data")
        print(f"   - Total Return: {metrics['Strategy']['Total Return']}")
        print(f"   - Sharpe Ratio: {metrics['Strategy']['Sharpe Ratio']}")
        print(f"   - Max Drawdown: {metrics['Strategy']['Max Drawdown']}")
        print(f"   - Excess Return vs VNINDEX: {metrics['Strategy']['Excess Return']}")
        
        print("\n✅ REAL data strategy completed successfully!")
        
        # Show plot
        if fig:
            plt.show()
        
    except Exception as e:
        print(f"❌ REAL data strategy failed: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
if __name__ == "__main__":
    main()